<a href="https://colab.research.google.com/github/l2ol3otic/AIM2127/blob/master/create_mockup_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. ติดตั้ง Library ที่จำเป็น**

In [ ]:
!pip install faker

**`2.สร้างข้อมูล Mockup ประกอบการ Workshop`**

In [ ]:
# prompt: create datafarme mockup idcard,name , surname thaipeople with faker th and merge data from 3province.xlsx with random row
import pandas as pd
from faker import Faker
import random
# Initialize Faker for Thai locale
fake = Faker('th_TH')

breed = ['ปทุมธานี 1', 'ชัยนาท 1', 'พิษณุโลก 2',' สุพรรณบุรี 1', 'สุพรรณบุรี 3', 'สุพรรณบุรี 60', 'ชัยนาท 80 (กข29)',
'ปทุมธานี 80 (กข31)', 'ขาวดอกมะลิ 105']  # Example rice varieties

# Number of records to generate
num_records = 150

# Generate fake data
id_cards = [fake.ssn() for _ in range(num_records)]
names = [fake.first_name() for _ in range(num_records)]
surnames = [fake.last_name() for _ in range(num_records)]
plant_area =   [random.randint(1,15) for _ in range(num_records)]
plant_product = [random.randint(200,700) for _ in range(num_records)]
cost = [random.randint(10000,180000) for _ in range(num_records)]
rice_varity = [random.choice(breed) for _ in range(num_records)]

# Create DataFrame
df_fake = pd.DataFrame({'idcard': id_cards, 'name': names, 'surname': surnames,'plant_area':plant_area,'plant_product':plant_product,'cost':cost,'rice_varity':rice_varity})

# Load Excel data (replace '3province.xlsx' with your actual file)
df_province = pd.read_excel('drive/MyDrive/workshop1/3province.xlsx')

# Randomly select rows from province data
num_rows_to_merge = 100  # Adjust as needed
random_indices = random.sample(range(0,150), num_rows_to_merge)
df_province_subset = df_province.iloc[random_indices]
#df_province_subset

# Merge DataFrames (adjust 'merge_column' if needed)
merged_df = pd.merge(df_fake, df_province_subset, left_index=True, right_index=True, how='left')
merged_df


**`3. สร้าง Error จำลอง`**

In [ ]:
# prompt: edit cell idcard with space some column and delete 1 number  some column

import random

# Function to introduce spaces and delete a number in an ID card string
def modify_idcard(idcard):
  # Randomly decide whether to modify
  if random.random() < 0.5:
    # Insert a space at a random position
    position = random.randint(1, len(idcard) - 1)
    idcard = idcard[:position] + ' ' + idcard[position:]

  if random.random() < 0.5:
    # Delete a random digit
    position = random.randint(0, len(idcard) - 1)
    idcard = idcard[:position] + idcard[position + 1:]

  return idcard

# Apply the modification to the 'idcard' column
merged_df['idcard'] = merged_df['idcard'].apply(modify_idcard)

merged_df


**`4. เชื่อมต่อ Google Drive`**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**`5. เชื่อมต่อ Google Sheet`**

In [ ]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=merged_df)

**`6. โหลดข้อมูลที่เราสร้างไว้จาก Google Sheet`**

In [ ]:
# prompt: read google sheet from url

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

# แก้ไข YOU_SHEET_ID เป็น google sheet id แต่ละคน
worksheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/YOU_SHEET_ID#gid=0').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
#print(rows)

# Convert to a DataFrame and render.
import pandas as pd
newDF = pd.DataFrame.from_records(rows)
newDF

**`7. สร้าง หัว Column ใหม่`**

In [ ]:
newDF.columns = newDF.iloc[0]  # Set the first row as headers
newDF = newDF[1:]  # Remove the original first row
newDF

**`8. แก้ Format เลขบัตรประชาชน`**

In [ ]:
# Remove spaces and hyphens from 'idcard' column
newDF['idcard'] = newDF['idcard'].str.replace(' ', '').str.replace('-', '')
# Update the Google Sheet
worksheet.update([newDF.columns.values.tolist()] + newDF.values.tolist())


**`9. ตรวจสอบความถูกต้องของเลขบัตรประชาชน`**

In [ ]:
# prompt: check idcard cell if lenght != 13 change idcard this column color to red

# Iterate through the 'idcard' column and check the length
for index, row in newDF.iterrows():
    idcard = row['idcard']
    goal_area = row['plant_area']

    if len(idcard) != 13:
        # Get the cell coordinates
        cell_range = f'A{index + 1}'  # +2 because DataFrame index starts from 0 and Google Sheets rows start from 1, and we have a header row
        # Change the background color to red
        print(cell_range)
        worksheet.format(cell_range, {'backgroundColor': {'green': 1}})

    if float(goal_area) > 10:
        # Get the cell coordinates
        cell_range = f'D{index + 1}'  # +2 because DataFrame index starts from 0 and Google Sheets rows start from 1, and we have a header row
        # Change the background color to red
        print(cell_range)
        worksheet.format(cell_range, {'backgroundColor': {'blue': 1}})

# Update the Google Sheet
worksheet.update([newDF.columns.values.tolist()] + newDF.values.tolist())
